In [1]:
import pandas as pd

# # 최대 행 수를 None으로 설정하여 모든 행이 출력되도록 함
# pd.set_option('display.max_rows', None)

# # 최대 열 수를 None으로 설정하여 모든 열이 출력되도록 함
# pd.set_option('display.max_columns', None)

# # 행과 열의 출력 옵션을 원래 상태로 되돌림
# pd.reset_option('display.max_rows')
# pd.reset_option('display.max_columns')

In [ ]:
# FILE_PATH = '.tmp/dataset/train_set.csv'
FILE_PATH = ".tmp/dataset/dataset_train.csv"
data = pd.read_csv(FILE_PATH)

data

In [ ]:
data.info()

# import Library


In [4]:
import matplotlib.pyplot as plt  # 시각화를 위한 Matplotlib 라이브러리
import pandas as pd  # 데이터 프레임을 다루기 위한 pandas 라이브러리
import seaborn as sns  # 시각화를 위한 Seaborn 라이브러리
from catboost import CatBoostClassifier  # CatBoost 분류 모델
from lightgbm import LGBMClassifier  # LightGBM 분류 모델
from sklearn.ensemble import RandomForestClassifier  # 랜덤 포레스트 분류 모델
from sklearn.linear_model import LogisticRegression  # 로지스틱 회귀 모델
from sklearn.metrics import roc_curve  # 모델 성능 평가를 위한 함수들
from sklearn.metrics import (
    accuracy_score,
    confusion_matrix,
    f1_score,
    precision_score,
    recall_score,
    roc_auc_score,
)
from sklearn.model_selection import (
    train_test_split,
)  # 데이터셋을 학습용과 검증용으로 나누기 위한 함수
from sklearn.neural_network import MLPClassifier  # MLP (다층 퍼셉트론) 분류 모델
from sklearn.preprocessing import (  # 데이터 스케일링을 위한 StandardScaler
    StandardScaler,
)
from xgboost import XGBClassifier  # XGBoost 분류 모델

# 데이터 처리 및 모델 학습 모듈화

> load_and_split_data: 데이터 로드와 학습/테스트 분할.

> scale_data: 학습 데이터와 테스트 데이터 스케일링.

> train_and_evaluate_model: 모델 학습 및 예측.

> evaluate_performance: 성능 지표 계산.

> plot_roc_curve: ROC-AUC 그래프 시각화.

> plot_confusion_matrix: 혼동 행렬 시각화.

> plot_feature_importance: 피처 중요도 시각화.

> run_model_pipeline 함수로 하나의 모델 실행 과정을 전체적으로 처리.


In [5]:
# 데이터 로드 및 분할 함수
def load_and_split_data(
    file_path, target_column="TARGET", test_size=0.3, random_state=42
):
    data = pd.read_csv(file_path)

    # 범주형 칼럼에 대해 One Hot Encoding 적용
    object_col = data.select_dtypes(include="object").columns
    ohe_df = pd.get_dummies(data, columns=object_col)
    bool_columns = ohe_df.select_dtypes(include="bool").columns
    ohe_df[bool_columns] = ohe_df[bool_columns].applymap(lambda x: 1 if x else 0)

    X = ohe_df.drop(columns=[target_column])
    y = ohe_df[target_column]

    return train_test_split(
        X, y, test_size=test_size, stratify=y, random_state=random_state
    )


# 데이터 스케일링 함수
def scale_data(X_train, X_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    return X_train_scaled, X_test_scaled


# 모델 학습 및 예측 함수
def train_and_evaluate_model(model, X_train_scaled, y_train, X_test_scaled, y_test):
    model.fit(X_train_scaled, y_train)
    y_pred_proba = model.predict_proba(X_test_scaled)[:, 1]
    y_pred = model.predict(X_test_scaled)
    return y_pred, y_pred_proba


# 성능 평가 함수
def evaluate_performance(y_test, y_pred, y_pred_proba):
    roc_auc = roc_auc_score(y_test, y_pred_proba)
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred)
    recall = recall_score(y_test, y_pred)
    f1 = f1_score(y_test, y_pred)

    print(f"AUC: {roc_auc:.4f}")
    print(f"Accuracy: {accuracy:.4f}")
    print(f"Precision: {precision:.4f}")
    print(f"Recall: {recall:.4f}")
    print(f"F1 Score: {f1:.4f}")

    return roc_auc


# ROC Curve 시각화 함수
def plot_roc_curve(y_test, y_pred_proba, roc_auc):
    fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)
    plt.figure(figsize=(8, 6))
    plt.plot(fpr, tpr, label="ROC Curve (area = %0.2f)" % roc_auc)
    plt.plot([0, 1], [0, 1], "k--")
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel("False Positive Rate")
    plt.ylabel("True Positive Rate")
    plt.title("Receiver Operating Characteristic (ROC) Curve")
    plt.legend(loc="lower right")
    plt.show()


# 혼동 행렬 시각화 함수
def plot_confusion_matrix(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues", cbar=False)
    plt.xlabel("Predicted Label")
    plt.ylabel("True Label")
    plt.title("Confusion Matrix")
    plt.show()


# 피처 중요도 시각화 함수
def plot_feature_importance(model, feature_names, top_n=3):
    if hasattr(model, "coef_"):
        importance = model.coef_[0]
    elif hasattr(model, "feature_importances_"):
        importance = model.feature_importances_
    else:
        print("Feature importance is not available for this model.")
        return

    importance_df = pd.DataFrame({"Feature": feature_names, "Importance": importance})
    importance_df = importance_df.sort_values(by="Importance", ascending=False)

    top_features = importance_df.head(top_n)
    bottom_features = importance_df.tail(top_n)

    plt.figure(figsize=(12, 8))

    plt.subplot(1, 2, 1)
    sns.barplot(x="Importance", y="Feature", data=top_features)
    plt.title(f"Top {top_n} Feature Importance")

    plt.subplot(1, 2, 2)
    sns.barplot(x="Importance", y="Feature", data=bottom_features)
    plt.title(f"Bottom {top_n} Feature Importance")

    plt.tight_layout()
    plt.show()


# 전체 실행 함수 (각 모델에 대해 호출할 수 있는 함수)
def run_model_pipeline(model, file_path, top_n=3):
    # 1. 데이터 로드 및 전처리, 분할
    X_train, X_test, y_train, y_test = load_and_split_data(file_path)

    # 2. 데이터 스케일링
    X_train_scaled, X_test_scaled = scale_data(X_train, X_test)

    # 3. 모델 학습 및 예측
    y_pred, y_pred_proba = train_and_evaluate_model(
        model, X_train_scaled, y_train, X_test_scaled, y_test
    )

    # 4. 성능 평가
    roc_auc = evaluate_performance(y_test, y_pred, y_pred_proba)

    # 5. ROC Curve 시각화
    plot_roc_curve(y_test, y_pred_proba, roc_auc)

    # 6. 혼동 행렬 시각화
    plot_confusion_matrix(y_test, y_pred)

    # 7. 피처 중요도 시각화
    plot_feature_importance(model, X_train.columns, top_n)

# Baseline Model Test


## 1. LogisticRegression


In [ ]:
# 로지스틱 회귀
logistic_model = LogisticRegression(max_iter=1000)
run_model_pipeline(logistic_model, FILE_PATH)

## 2. RandomForest


In [ ]:
# 랜덤 포레스트
rf_model = RandomForestClassifier(
    n_estimators=100, random_state=42, class_weight="balanced"
)
run_model_pipeline(rf_model, FILE_PATH)

## 3. XGBoost


In [ ]:
X_train, X_test, y_train, y_test = load_and_split_data(FILE_PATH)
X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
scale_pos_weight = len(y_train) / sum(y_train)  # 양성 클래스 비율 계산
xgb_model = XGBClassifier(
    use_label_encoder=False, eval_metric="logloss", scale_pos_weight=scale_pos_weight
)

run_model_pipeline(xgb_model, FILE_PATH)

## 4. LightGBM


In [ ]:
X_train, X_test, y_train, y_test = load_and_split_data(FILE_PATH)
X_train_scaled, X_test_scaled = scale_data(X_train, X_test)
lgb_model = LGBMClassifier(class_weight="balanced", random_state=42)

run_model_pipeline(lgb_model, FILE_PATH)

## 5. CatBoost


In [ ]:
CB_model = CatBoostClassifier(
    iterations=1000,
    learning_rate=0.1,
    depth=6,
    loss_function="Logloss",
    eval_metric="AUC",
    class_weights=[1, len(y_train) / sum(y_train)],
    random_state=42,
    verbose=100,
)

run_model_pipeline(CB_model, FILE_PATH)

## 6. MLP


In [ ]:
MLP_model = MLPClassifier(hidden_layer_sizes=(100,), max_iter=500, random_state=42)
run_model_pipeline(MLP_model, FILE_PATH)